# Лабораторна робота №4

Варіант 19
Терещенко Олексій, група ДА-21


## 2. Постановка задачі

Маємо множину кандидатів (альтернатив)
\( X = \{a, b, c, d\} \).  
Профіль індивідуальних переваг задається у вигляді таблиці:

| Кількість виборців | 3   | 4   | 4   |
| ------------------ | --- | --- | --- |
| 1‑ша позиція       | c   | b   | a   |
| 2‑га позиція       | d   | a   | b   |
| 3‑тя позиція       | a   | d   | c   |
| 4‑та позиція       | b   | c   | d   |

Це означає:

- 3 виборці мають порядок уподобань: c ≻ d ≻ a ≻ b;
- 4 виборці мають порядок: b ≻ a ≻ d ≻ c;
- 4 виборці мають порядок: a ≻ b ≻ c ≻ d.

Загальна кількість виборців:  
\(3 + 4 + 4 = 11\).


## 3. Кодування профілю


In [ ]:

candidates = ["a", "b", "c", "d"]


profile = [
    (3, ["c", "d", "a", "b"]),  # 3 виборці: c ≻ d ≻ a ≻ b
    (4, ["b", "a", "d", "c"]),  # 4 виборці: b ≻ a ≻ d ≻ c
    (4, ["a", "b", "c", "d"]),  # 4 виборці: a ≻ b ≻ c ≻ d
]

print("Кандидати:", candidates)
print("Профіль виборців:")
for count, ranking in profile:
    print(f"  {count} виборців:", " ≻ ".join(ranking))


Кандидати: ['a', 'b', 'c', 'd']
Профіль виборців:
  3 виборців: c ≻ d ≻ a ≻ b
  4 виборців: b ≻ a ≻ d ≻ c
  4 виборців: a ≻ b ≻ c ≻ d


## 4. Метод Борда


In [ ]:
from collections import defaultdict

def borda_scores(profile, candidates):
    m = len(candidates)
    scores = defaultdict(int)

    for count, ranking in profile:
        for position, cand in enumerate(ranking):
            points = (m - 1) - position
            scores[cand] += count * points

    return dict(scores)

borda = borda_scores(profile, candidates)

print("Бали за методом Борда:")
for c in sorted(candidates):
    print(f"  {c}: {borda[c]}")

print("\nРанжування (від кращого до гіршого):")
for c, s in sorted(borda.items(), key=lambda x: -x[1]):
    print(f"  {c} (сума балів = {s})")

Бали за методом Борда:
  a: 23
  b: 20
  c: 13
  d: 10

Ранжування (від кращого до гіршого):
  a (сума балів = 23)
  b (сума балів = 20)
  c (сума балів = 13)
  d (сума балів = 10)


## 5. Попарні порівняння та матриця більшості


In [ ]:

def pairwise_matrix(profile, candidates):
    N = defaultdict(int)

    for count, ranking in profile:
        pos = {cand: i for i, cand in enumerate(ranking)}
        for a in candidates:
            for b in candidates:
                if a == b:
                    continue
                if pos[a] < pos[b]:
                    N[(a, b)] += count
    return N

N = pairwise_matrix(profile, candidates)

print("Попарні порівняння N(a, b):")
for i, a in enumerate(candidates):
    for b in candidates[i+1:]:
        print(f"  {a} проти {b}: N({a},{b}) = {N[(a,b)]},  N({b},{a}) = {N[(b,a)]}")


Попарні порівняння N(a, b):
  a проти b: N(a,b) = 7,  N(b,a) = 4
  a проти c: N(a,c) = 8,  N(c,a) = 3
  a проти d: N(a,d) = 8,  N(d,a) = 3
  b проти c: N(b,c) = 8,  N(c,b) = 3
  b проти d: N(b,d) = 8,  N(d,b) = 3
  c проти d: N(c,d) = 7,  N(d,c) = 4


## 6. Метод Копленда


In [ ]:

def copeland_scores(N, candidates):
    scores = {c: 0 for c in candidates}
    for i, a in enumerate(candidates):
        for b in candidates[i+1:]:
            nab = N[(a, b)]
            nba = N[(b, a)]
            if nab > nba:
                scores[a] += 1
                scores[b] -= 1
            elif nba > nab:
                scores[b] += 1
                scores[a] -= 1
    return scores

copeland = copeland_scores(N, candidates)

print("Оцінки Копленда:")
for c in sorted(candidates):
    print(f"  {c}: {copeland[c]}")

print("\nРанжування (від кращого до гіршого):")
for c, s in sorted(copeland.items(), key=lambda x: -x[1]):
    print(f"  {c} (оцінка Копленда = {s})")


Оцінки Копленда:
  a: 3
  b: 1
  c: -1
  d: -3

Ранжування (від кращого до гіршого):
  a (оцінка Копленда = 3)
  b (оцінка Копленда = 1)
  c (оцінка Копленда = -1)
  d (оцінка Копленда = -3)


## 7. Правило паралельного виключення


In [ ]:

def majority_winner(a, b, N):
    nab = N[(a, b)]
    nba = N[(b, a)]
    if nab > nba:
        return a
    elif nba > nab:
        return b
    else:
        return a

def parallel_elimination(order, N):
    a, b, c, d = order
    winner1 = majority_winner(a, b, N)
    winner2 = majority_winner(c, d, N)
    final_winner = majority_winner(winner1, winner2, N)
    return winner1, winner2, final_winner

order = ("a", "b", "c", "d")
w1, w2, wf = parallel_elimination(order, N)

print("Паралельне виключення для порядку (a, b, c, d):")
print("  Півфінал 1:", w1, "— переможець пари (a, b)")
print("  Півфінал 2:", w2, "— переможець пари (c, d)")
print("  Фінал:", wf, "— переможець між", w1, "та", w2)


Паралельне виключення для порядку (a, b, c, d):
  Півфінал 1: a — переможець пари (a, b)
  Півфінал 2: c — переможець пари (c, d)
  Фінал: a — переможець між a та c
